# Label side

In [1]:
import numpy as np
from jesse import helpers, research

_, candles = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2022-07-01"),
    helpers.date_to_timestamp("2025-10-10"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

candles.shape

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.12/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


(1723680, 6)

In [2]:
np.save("data/btc_1m.npy", candles)
del candles

In [ ]:
import numpy as np

candles = np.load("data/btc_1m.npy")
candles.shape

In [ ]:
from src.bars.fusion.demo import DemoBar

bar_container = DemoBar(max_bars=-1)
bar_container.update_with_candles(candles)
merged_bar = bar_container.get_fusion_bars()
merged_bar.shape

In [ ]:
from jesse import utils

df_plot_bar = utils.numpy_candles_to_dataframe(
    merged_bar, name_date="time"
)
df_plot_bar.head(1)

In [ ]:
from lightweight_charts import Chart

chart = Chart()
# chart = Chart(inner_width=1, inner_height=0.8)
# chart2 = chart.create_subchart(position="bottom", width=1, height=0.2, sync=True)

chart.set(df_plot_bar[["time", "open", "high", "low", "close", "volume"]])

# line = chart2.create_line(name="zigzag", style="dotted")
# line.set(df_plot_bar[["time", "zigzag"]])

chart.show(block=True)

In [ ]:
from research.labeler.gmm_labeler import GMMLabeler

labeler_L4 = GMMLabeler(merged_bar, 4)
labeler_L4.plot_label_on_candles()

In [ ]:
from research.labeler.gmm_labeler import GMMLabeler

labeler_L5 = GMMLabeler(merged_bar, 5)
labeler_L5.plot_label_on_candles()

In [ ]:
# labeler_L5.plot_label_returns()

In [ ]:
labeler_L6 = GMMLabeler(merged_bar, 6)
labeler_L6.plot_label_on_candles()

In [ ]:
labeler_L7 = GMMLabeler(merged_bar, 7)
labeler_L7.plot_label_on_candles()

In [ ]:
for labeler, idx in zip([labeler_L4, labeler_L5, labeler_L6, labeler_L7], [4, 5, 6, 7]):
    label_hard = labeler.label_hard_state
    print(f"{label_hard.shape = }")
    np.save(f"data/raw_label_hard_L{idx}.npy", label_hard)
    label_direction_force = labeler.label_direction_force
    print(f"{label_direction_force.shape = }")
    np.save(f"data/raw_label_direction_L{idx}.npy", label_direction_force)

# Label meta

In [ ]:
import numpy as np
import pandas as pd
from strategies.BinanceBtcEntropyBarV1.config import SIDE
from strategies.BinanceBtcEntropyBarV1.config import get_side_model

side_model = get_side_model(False)

merged_bar = np.load("data/merged_bar.npy")

df_features = pd.read_parquet("data/features.parquet")

side_res = side_model.predict(df_features[SIDE])

side_pred_label = np.where(side_res > 0.5, 1, -1)
print(side_pred_label.shape)

print(np.unique(side_pred_label, return_counts=True))

In [ ]:
close_prices = merged_bar[:, 2]
len_gap = len(close_prices) - len(side_pred_label)
close_prices = close_prices[len_gap - 1 : -1]

assert len(close_prices) == len(side_pred_label)

In [ ]:
meta_label = np.zeros(len(side_pred_label))

TRADE_FEE = 0.05 / 100

start_idx = 0
cumsum_ret = 0
start_price = 0
for idx, (i, p) in enumerate(zip(side_pred_label, close_prices)):
    if i == 1 or i == -1:
        if idx == 0:
            # 开始持仓
            start_idx = idx
            start_price = p
            cumsum_ret -= TRADE_FEE
        elif side_pred_label[idx - 1] != i:
            # 反向持仓，先结算收益
            cumsum_ret -= TRADE_FEE
            cumsum_ret += np.log(p / start_price) * side_pred_label[idx - 1]
            if cumsum_ret > 0:
                meta_label[start_idx:idx] = 1
            cumsum_ret = 0
            start_price = p
            start_idx = idx
            cumsum_ret -= TRADE_FEE
        else:
            # 继续持仓
            continue
    else:
        raise ValueError(f"side_pred_label[{idx}] = {i} is not valid")
else:
    last_price = merged_bar[-1, 2]
    # 结算最后一根bar的持仓, 可能还没有结算，所以先不加trade fee
    if i == side_pred_label[idx - 1]:
        # 已经开仓，结算
        cumsum_ret += np.log(last_price / start_price) * side_pred_label[idx - 1]
        if cumsum_ret > 0:
            meta_label[start_idx:] = 1
    else:
        # 反向开仓
        cumsum_ret -= TRADE_FEE
        cumsum_ret += np.log(last_price / start_price) * side_pred_label[idx - 1]
        if cumsum_ret > 0:
            meta_label[start_idx:] = 1


print(np.unique(meta_label, return_counts=True))

In [ ]:
np.save("data/label_meta.npy", meta_label)